# Mervar - Kohonen Maps
### Alexander Mervar
### Professor Brown
### COGS-Q 355 SP22
### 2.9.2022

Consider a Kohonen self organizing feature map (SOFM), as described in Kohonen (1982). Consider a map with 2500 elements arranged in a 50x50 rectangular grid (matrix). Each input vector has 3 elements in the continuous range of [0,1] that correspond to a signal of one of the primary colors (red, green, blue). For example, (1, 0, 0) is an input pattern that corresponds to the color red.

Implement a simple SOFM in python following the equations described in Kohonen (1982), but simplified as follows. Assume a weight matrix W of dimensions 50x50x3, that connects each input to each SOFM unit. The input to each SOFM unit is the external input plus the lateral input:
- External Input = W * inputs (Equation 4 in Kohonen (1982))
- Activity from A (from perspective of B):
  - a * 8 ; If euclidean distance is between A and B < 3
  - a * -1 ; if euclidean distance is between A and B is > 3 < 8
  - 0 otherwise
- The total activity $\eta$ ("eta") of each SOFM is a piece-wise sigmoid
  - 0 if the sum of the external and lateral input is <= 0
  - 5 if the sum of the external and lateral input is >= 5
  - The sum of the external and lateral input, otherwise
- Weights W from the external inputs to the SOFM units are adjusted by the following rule
  - W(t+1) = (W(t) + learning*eta*inputs)/(the euclidean length of the input weights to the SOFM unit)
  - learning = 0.01

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from math import dist

fig, ax = plt.subplots(figsize=(1,1))

# Define Learning Rate
learningRate = 0.01

# Define weight matrix size
xSixe = 50
ySize = 50
weightVectorSize = 3

# Define Frequency of Input Vectors
redFrequency = 0.3333
greenFrequency = 0.6666
blueFrequency  = 1 - greenFrequency


# Define an random X*Y*WV array
weightMatrix = np.random.random((xSize, ySize, weightVectorSize))

# Traverse the matrix
for i in range(50):
    #
    for j in range(50):
        # Define the input vector
        for k in range(3):
            # 

plt.imshow(weightMatrix)

plt.show()

# ax.set_title('%f'%(count))

fig.canvas.draw()

fig.canvas.flush_events()